In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
# We won't need TensorFlow here
# %pip uninstall -y tensorflow
# # Install `transformers` from master
# %pip install git+https://github.com/huggingface/transformers
# %pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

Note: you may need to restart the kernel to use updated packages.


  Cloning https://github.com/huggingface/transformers to c:\temp\pip-req-build-x8oju_o3
  Resolved https://github.com/huggingface/transformers to commit acd653164b6874e395ac9d46850f67599d8cdb58
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Obtaining dependency information for huggingface-hub<1.0,>=0.19.3 from https://files.pythonhosted.org/packages/05/09/1945ca6ba3ad8ad6e2872ba682ce8d68c5e63c8e55458ed8ab4885709f1d/huggingface_hub-0.19.4-py3-none-any.whl.metadata
  Obtaining dependency information for tokenizers<0.19,>=0.14 from https://files.pythonhosted.org/packages/42/db/0061fb8004ce9173b9249a0c323c799be51f2c8e6d4ff3cc38b549c3f8b6/tokenizers-0.15.0-cp310-none-win_amd64.whl.metadata
  Obtaining dependenc

  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers 'C:\Temp\pip-req-build-x8oju_o3'
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2023.4.0 requires fsspec==2023.4.0, but you have fsspec 2023.12.1 which is incompatible.


Note: you may need to restart the kernel to use updated packages.


'grep' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "BPEtokenizer/vocab.json",
    "BPEtokenizer/merges.txt",
)

In [6]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [7]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=50_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [8]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast("BPEtokenizer/vocab.json",
                                 "BPEtokenizer/merges.txt", 
                                 max_len=512)

In [9]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [10]:
model.num_parameters()
# => 84 million parameters

81966416

In [8]:
%%time
# from transformers import LineByLineTextDataset

# dataset = LineByLineTextDataset(
#     tokenizer=tokenizer,
#     file_path=r"C:\Users\Jonathan Ipe\Desktop\unlabeled_data_sentences.txt",
#     block_size=128,
# )
from datasets import load_dataset

dataset = load_dataset("text", data_files=r"unlabeled_data_sentences.txt", split='train')
shuffled_dataset = dataset.shuffle(seed=42, buffer_size=10_000)

train_dataset = shuffled_dataset.take(25_000)
eval_dataset = shuffled_dataset.skip(25_000)

Found cached dataset text (file://C:/Users/Jonathan Ipe/.cache/huggingface/datasets/text/default-9260811a8c434195/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)


NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

In [14]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [18]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="BERT",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [19]:
%%time
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/jipe/anaconda3/envs/E208/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


CPU times: user 4min 11s, sys: 2.25 s, total: 4min 13s
Wall time: 2min 27s


TrainOutput(global_step=313, training_loss=4.585713249425918, metrics={'train_runtime': 147.479, 'train_samples_per_second': 271.259, 'train_steps_per_second': 2.122, 'total_flos': 1326375701360640.0, 'train_loss': 4.585713249425918, 'epoch': 1.0})

In [20]:
trainer.save_model("composer-classification")

In [ ]:
# from tokenizers.implementations import ByteLevelBPETokenizer
# from tokenizers.processors import BertProcessing
# from torch.utils.data import Dataset

# class ComposerClassificationDataset(Dataset):
#     def __init__(self, evaluate: bool = False):
#         tokenizer = ByteLevelBPETokenizer(
#             "note-classifier-vocab.json",
#             "note-classifier-merges.txt",
#         )
#         tokenizer._tokenizer.post_processor = BertProcessing(
#             ("</s>", tokenizer.token_to_id("</s>")),
#             ("<s>", tokenizer.token_to_id("<s>")),
#         )
#         tokenizer.enable_truncation(max_length=512)
#         # or use the RobertaTokenizer from `transformers` directly.

#         self.examples = []

#         src_files = 'sentences.txt'
#         for src_file in src_files:
#             print("🔥", src_file)
#             lines = src_file.read_text(encoding="utf-8").splitlines()
#             self.examples += [x.ids for x in tokenizer.encode_batch(lines)]

#     def __len__(self):
#         return len(self.examples)

#     def __getitem__(self, i):
#         # We’ll pad at the batch level.
#         return torch.tensor(self.examples[i])